In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import Sequential, utils
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# add channel to images (required by tensorflow)
img_shape=(28, 28, 1)
X_train = X_train.reshape(X_train.shape[0], *img_shape)
X_test = X_test.reshape(X_test.shape[0], *img_shape)

# convert images to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# scale data between [0,1]
X_train /= 255
X_test /= 255

# convert labels to categorical
num_classes = 10
y_train_cat = utils.to_categorical(y_train, num_classes)
y_test_cat = utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=img_shape))

model.add(Conv2D(64, kernel_size=3, strides=2, activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

# predict class
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

In [ ]:
# remove previous logs
%rm -r logs/ 2> /dev/null

# create callback to store history
tb = TensorBoard(log_dir='logs', histogram_freq=1)
tb.set_model(model)

In [ ]:
epochs = 10
batch_size = 128
h = model.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test_cat), callbacks=[tb])

In [ ]:
plt.figure(figsize=(20, 8))
fontsize = 12

plt.subplot(1, 2, 1)
plt.plot(h.history['loss'], label='Train')
plt.plot(h.history['val_loss'], label='Validation')
plt.xlabel('Epoch', fontsize=fontsize)
plt.ylabel('Loss', fontsize=fontsize)
plt.legend()
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(h.history['accuracy'], label='Train')
plt.plot(h.history['val_accuracy'], label='Validation')
plt.xlabel('Epoch', fontsize=fontsize)
plt.ylabel('Accuracy', fontsize=fontsize)
plt.legend()
plt.grid()

In [ ]:
# find first occurrence of each digit in test data and predict class
index = [np.where(y_test == i)[0][0] for i in range(num_classes)]
imgs = X_test[index]
digits = model.predict_classes(imgs)

rows = 2
cols = 5
fig, axs = plt.subplots(rows, cols, figsize=(20, 8))

# plot each image and display the predicted class
for i in range(rows):
    for j in range(cols):
        index = j + i*cols
        axs[i,j].imshow(imgs[index].reshape(img_shape[0:2]), cmap='gray_r');
        axs[i,j].set_title(f'Predicted Class: {digits[index]}')

In [ ]:
# get accuracy on test data by digit
digits = range(num_classes)
index = [y_test == i for i in digits]
acc = [model.evaluate(X_test[i], y_test_cat[i], verbose=0)[1] for i in index]

plt.figure(figsize=(10, 6))
plt.bar(digits, acc)
plt.xlabel('Digit', fontsize=fontsize)
plt.ylabel('Accuracy', fontsize=fontsize)
plt.xticks(digits, fontsize=fontsize);